In [1]:
!pip install -U sentence-transformers

import pandas as pd
import numpy as np

from tqdm import tqdm

from collections import Counter

import gc
import cloudpickle

import os, sys
import random

import warnings
from sentence_transformers import SentenceTransformer, util


import xgboost as xgb

print('Using \033[34mXGBoost', xgb.__version__, '\033[0m')

     |████████████████████████████████| 78 kB 690 kB/s 
     |████████████████████████████████| 3.1 MB 1.6 MB/s 
     |████████████████████████████████| 61 kB 352 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=07f50761731ed23e50eebd6d1d3f15191de08cb98f9895c6fb6e7dc826f3b95e
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
model = SentenceTransformer('clip-ViT-B-32')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [3]:
matchings = pd.read_csv('../input/test-combiner-target-dataset/final_matchings.csv', keep_default_na=False)

In [4]:
def make_embeddings(field, n=None):
    embeddings = []
    
    if n is None:
        n = matchings.shape[0]
    
    bads = 0

    for k in range(n):
        v = matchings.loc[k, field]
        
        try:
            e = model.encode(v, show_progress_bar=False)
            embeddings.append(e)
        except:
            embeddings.append(np.full(512, 1))
            bads += 1
        
    print('BADS: {0:0.3f}'.format(bads / n))
    
    return np.vstack(embeddings)

In [5]:
%%time

for field in ['final_caption', 'final_page_title']:
    e = make_embeddings(field)

    with open('{}_embeddings.data'.format(field), 'wb') as file:
        cloudpickle.dump(e, file)

Token indices sequence length is longer than the specified maximum sequence length for this model (116 > 77). Running this sequence through the model will result in indexing errors


BADS: 0.003
BADS: 0.000
CPU times: user 3h 13min 15s, sys: 57.9 s, total: 3h 14min 13s
Wall time: 1h 37min 11s
